In [171]:
import json
import pennylane as qml
import pennylane.numpy as np

dev = qml.device("default.qubit", wires=5)


@qml.qnode(dev)
def evolve_state(coeffs, time):
    """
    Args:
        coeffs (list(float)): A list of the coupling constants g_1, g_2, g_3, and g_4
        time (float): The evolution time of th system under the given Hamiltonian

    Returns:
        (numpy.tensor): The density matrix for the evolved state of the central spin.
    """

    # We build the Hamiltonian for you

    operators = [
        qml.PauliZ(0) @ qml.PauliZ(1),
        qml.PauliZ(0) @ qml.PauliZ(2),
        qml.PauliZ(0) @ qml.PauliZ(3),
        qml.PauliZ(0) @ qml.PauliZ(4),
    ]
    hamiltonian = qml.Hamiltonian(coeffs, operators)


    # Put your code here #
    qml.RX(np.pi / 2, wires=0)
    qml.RX(0.4, wires=1)
    qml.RX(1.2, wires=2)
    qml.RX(1.8, wires=3)
    qml.RX(0.6, wires=4)
    qml.templates.ApproxTimeEvolution(hamiltonian, time, 10)

    # Return the required density matrix.
    return qml.density_matrix(wires=0)


def purity(rho):
    """
    Args:
        rho (array(array(complex))): An array-like object representing a density matrix

    Returns:
        (float): The purity of the density matrix rho

    """


    # Put your code here
    # Return the purity
    return np.trace(np.dot(rho, rho))


def recoherence_time(coeffs):
    """
    Args:
        coeffs (list(float)): A list of the coupling constants g_1, g_2, g_3, and g_4.

    Returns:
        (float): The recoherence time of the central spin.

    """
    upper = np.pi * 10
    error = 5e-2
    num_steps = 5 * int(upper / error)
    print(num_steps)
    steps = np.linspace(0, upper, num_steps)
    found = False
    window = [purity(evolve_state(coeffs, t)) for t in steps[:2]]
    window.append(0)
    for i in range(2, num_steps - 3):
        window[2] = purity(evolve_state(coeffs, steps[i]))
        if (abs(window[1] - 1.0) > 1e-2) or (window[1] < window[0]) or (window[1] < window[2]):
            window[0] = window[1]
            window[1] = window[2]
            continue
        found = True
        break
    print(window)
    assert found
    return steps[i]



In [172]:
from matplotlib import pyplot as plt
rec_time = recoherence_time([5, 5, 5, 5])
print(rec_time)


3140
[(0.9783246796027147+0j), (0.9982603341934984+0j), (0.995754238956585+0j)]
0.3202643036472551


In [173]:

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    params = json.loads(test_case_input)
    output = recoherence_time(params)

    return str(output)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    print(solution_output, expected_output)
    assert np.isclose(solution_output, expected_output, rtol=5e-2)


# These are the public test cases
test_cases = [
    ('[5,5,5,5]', '0.314'),
    ('[1.1,1.3,1,2.3]', '15.71')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[5,5,5,5]'...
3140
[(0.9783246796027147+0j), (0.9982603341934984+0j), (0.995754238956585+0j)]
0.3202643036472551 0.314
Correct!
Running test case 1 with input '[1.1,1.3,1,2.3]'...
3140
[(0.999785371377115+0j), (0.9997853713771189+0j), (0.9980720968819021+0j)]
15.722975657182431 15.71
Correct!


In [81]:
np.trace

<function autograd.tracer.primitive.<locals>.f_wrapped(*args, **kwargs)>

In [154]:
5e-2

0.05

In [156]:
10*np.pi / 5e-2

628.3185307179585